<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/PULP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [32]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

def standardize_column_names(df):
    def standardize(col):
        col_lower = col.lower()
        if "player" in col_lower:
            return "Player"
        elif "avg" in col_lower:
            return "AVG"
        else:
            return col
    df.columns = [standardize(col) for col in df.columns]
    return df

df = standardize_column_names(df)

k = 0.02
df['Value'] = 1000 * np.exp(-k * (df['AVG'] - 1))
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)
df

,Rank,Player,Team,Bye,POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG,Value
0,1.0,Christian McCaffrey,SF,9,RB,1.0,1.0,1.0,1.0,NaN,1.0,1000.000000
1,2.0,CeeDee Lamb,DAL,7,WR,3.0,3.0,2.0,2.0,NaN,2.6,968.506582
2,3.0,Tyreek Hill,MIA,6,WR,4.0,2.0,3.0,3.0,NaN,3.2,956.953957
3,4.0,Bijan Robinson,ATL,12,RB,2.0,6.0,8.0,4.0,NaN,5.0,923.116346
4,5.0,Breece Hall,NYJ,12,RB,5.0,8.0,7.0,5.0,NaN,5.4,915.760877
...,...,...,...,...,...,...,...,...,...,...,...,...
937,945.0,Mitchell Tinsley,CIN,12,WR,NaN,NaN,895.0,NaN,NaN,895.0,0.000017
938,947.0,Anthony Gould,IND,14,WR,NaN,NaN,899.0,NaN,NaN,899.0,0.000016
939,948.0,Patrick Murtagh,NaN,NaN,TE,NaN,NaN,900.0,NaN,NaN,900.0,0.000016
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df = df[['Player', 'POS', 'AVG', 'Value']].dropna()

# Only keep relevant positions
df = df[df['POS'].str.contains('QB|RB|WR|TE|K|DST', na=False)]

print(f'Length of df: {len(df)}')
print(f'value count of POS: {df["POS"].value_counts()}')
print(f'df head: {df.head(20)}')

Length of df: 940
value count of POS: POS
WR     333
RB     219
TE     165
QB     127
K       64
DST     32
Name: count, dtype: int64
df head:                  Player POS   AVG        Value
0   Christian McCaffrey  RB   1.0  1000.000000
1           CeeDee Lamb  WR   2.6   968.506582
2           Tyreek Hill  WR   3.2   956.953957
3        Bijan Robinson  RB   5.0   923.116346
4           Breece Hall  RB   5.4   915.760877
5     Amon-Ra St. Brown  WR   6.2   901.225297
6         Ja'Marr Chase  WR   6.6   894.044258
7      Justin Jefferson  WR   7.0   886.920437
8        Saquon Barkley  RB   9.2   848.742022
9            A.J. Brown  WR  10.2   831.935804
10      Jonathan Taylor  RB  10.4   828.614707
11       Garrett Wilson  WR  12.4   796.124260
12         Jahmyr Gibbs  RB  12.8   789.780674
13           Puka Nacua  WR  14.2   767.973540
14       Kyren Williams  RB  16.0   740.818221
15  Marvin Harrison Jr.  WR  16.8   729.059450
16   Travis Etienne Jr.  RB  17.4   720.363020
17        D

In [ ]:
df_small = df.nlargest(220, 'Value').copy()

print(f'Length of df_small: {len(df_small)}')
print(df_small.head(20))

print(df_small['Player'].duplicated().sum())
print(df_small['POS'].value_counts())
print(df_small.head(20))

In [4]:
!pip install pulp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 47.5 MB/s eta 0:00:00


# MIP, no Draft order

In [5]:
number_of_teams = 8
teams = list(range(1, number_of_teams + 1))
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

players = df.index.tolist()

# Modell erstellen
model = LpProblem("Fantasy_Football_Draft", LpMaximize)

# Entscheidungsvariablen x[t,i]
x = LpVariable.dicts("x", ((t, i) for t in teams for i in players), cat=LpBinary)

# Zielfunktion: Maximiere den Gesamtwert aller Teams
model += lpSum(df.loc[i, "Value"] * x[t, i] for t in teams for i in players)

# 1. Jeder Spieler kann nur von einem Team ausgewählt werden
for i in players:
    model += lpSum(x[t, i] for t in teams) <= 1

# 2. Teamgröße: maximal 15 Spieler pro Team
for t in teams:
    model += lpSum(x[t, i] for i in players) <= max_team_size

# 3. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(x[t, i] for i in players if df.loc[i, "POS"] == pos) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=0)
model.solve(solver)

# Ergebnis ausgeben
result = {}
for t in teams:
    team_players = [i for i in players if x[t, i].varValue == 1]
    result[t] = df.loc[team_players, ["Player", "POS", "Value"]].reset_index(drop=True)

# Ausgabe: result ist ein Dictionary mit Teamnummer als Key und DataFrame als Value
for t in teams:
    print(f"\nTeam {t}:")
    print(result[t])



Team 1:
                 Player  POS        Value
0   Christian McCaffrey   RB  1000.000000
1          Jahmyr Gibbs   RB   789.780674
2         Davante Adams   WR   657.046820
3         De'Von Achane   RB   589.783358
4         Brandon Aiyuk   WR   516.851334
5           Cooper Kupp   WR   508.647519
6            DK Metcalf   WR   447.535238
7         Jaylen Waddle   WR   435.178059
8          Amari Cooper   WR   327.587528
9           Jordan Love   QB   244.632058
10         Keenan Allen   WR   208.461689
11          David Njoku   TE   189.379943
12     Christian Watson   WR   132.125903
13        Justin Tucker    K   116.717359
14  Pittsburgh Steelers  DST    42.766508

Team 2:
                    Player  POS       Value
0              CeeDee Lamb   WR  968.506582
1              Tyreek Hill   WR  956.953957
2        Amon-Ra St. Brown   WR  901.225297
3         Justin Jefferson   WR  886.920437
4           Kyren Williams   RB  740.818221
5      Marvin Harrison Jr.   WR  729.059450
6 

# MIP, Snake Draft

In [6]:
# Parameter
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
players = df.index.tolist()
num_picks = max_team_size * number_of_teams

# Snake-Draft-Reihenfolge erzeugen
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]
# draft_order[p] = Team, das bei Pick p dran ist

# MIP-Modell
model = LpProblem("Fantasy_Snake_Draft", LpMaximize)

# Entscheidungsvariablen: x[t, i, p] = 1, wenn Team t Spieler i bei Pick p wählt
x = LpVariable.dicts(
    "x",
    ((t, i, p) for t in teams for i in players for p in range(num_picks)),
    cat=LpBinary
)

# Zielfunktion: Summe der Werte aller gezogenen Spieler maximieren
model += lpSum(
    df.loc[i, "Value"] * x[t, i, p]
    for t in teams for i in players for p in range(num_picks)
)

# 1. Jeder Pick wird von genau einem Team gemacht, und das nur in der Draft-Reihenfolge
for p in range(num_picks):
    t = draft_order[p]
    model += lpSum(x[t, i, p] for i in players) == 1
    # Alle anderen Teams dürfen bei diesem Pick niemanden wählen
    for t2 in teams:
        if t2 != t:
            model += lpSum(x[t2, i, p] for i in players) == 0

# 2. Jeder Spieler wird höchstens einmal gewählt
for i in players:
    model += lpSum(x[t, i, p] for t in teams for p in range(num_picks)) <= 1

# 3. Jedes Team wählt genau max_team_size Spieler
for t in teams:
    model += lpSum(x[t, i, p] for i in players for p in range(num_picks)) == max_team_size

# 4. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(
            x[t, i, p]
            for i in players if df.loc[i, "POS"] == pos
            for p in range(num_picks)
        ) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=1)
model.solve(solver)

# Ergebnisse sammeln: Für jeden Pick, wer wurde von wem gezogen?
draft_results = []
for p, t in enumerate(draft_order):
    for i in players:
        if x[t, i, p].varValue == 1:
            draft_results.append({
                "Pick": p + 1,
                "Team": t,
                "Player": df.loc[i, "Player"],
                "POS": df.loc[i, "POS"],
                "Value": df.loc[i, "Value"]
            })

draft_df = pd.DataFrame(draft_results)
print(draft_df)

# Optional: Team-Zusammenfassungen ausgeben
for t in teams:
    team_picks = draft_df[draft_df["Team"] == t]
    print(f"\nTeam {t}:")
    print(team_picks[["Pick", "Player", "POS", "Value"]].reset_index(drop=True))


    Pick  Team               Player  POS        Value
0      1     1       Garrett Wilson   WR   796.124260
1      2     2       Kyren Williams   RB   740.818221
2      3     3          Breece Hall   RB   915.760877
3      4     4         Stefon Diggs   WR   431.710523
4      5     4          Cooper Kupp   WR   508.647519
5      6     3        Isiah Pacheco   RB   681.131427
6      7     2         Malik Nabers   WR   385.968314
7      8     1           A.J. Brown   WR   831.935804
8      9     1        DeVonta Smith   WR   423.162082
9     10     2         Travis Kelce   TE   616.313202
10    11     3       Bijan Robinson   RB   923.116346
11    12     4           Mike Evans   WR   546.620774
12    13     4  Christian McCaffrey   RB  1000.000000
13    14     3        De'Von Achane   RB   589.783358
14    15     2   Patrick Mahomes II   QB   571.209064
15    16     1         Drake London   WR   611.402366
16    17     1     Deebo Samuel Sr.   WR   537.944438
17    18     2  Michael Pitt

# MIP: Greedy Snake Draft

In [7]:
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in min_pos} for t in teams}

# Snake-Draft-Reihenfolge
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    # Team-Needs bestimmen
    needs = []
    for pos, min_c in min_pos.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    # Wenn alle Mindestanforderungen erfüllt, alle Positionen zulassen
    if not needs:
        needs = list(min_pos.keys())
    # Wähle den besten verfügbaren Spieler, der eine Need-Position abdeckt
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available  # Falls keine Need mehr, beliebiger Spieler
    best_idx = candidates["Value"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren
    rosters[t].append(best_player["Player"])
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"]
    })
    available = available.drop(best_idx)
    # Team voll? Dann überspringen in Zukunft (optional)

draft_df = pd.DataFrame(draft_results)
print(draft_df)


    Pick  Team               Player  POS        Value
0      1     1  Christian McCaffrey   RB  1000.000000
1      2     2          CeeDee Lamb   WR   968.506582
2      3     3          Tyreek Hill   WR   956.953957
3      4     4       Bijan Robinson   RB   923.116346
4      5     4          Breece Hall   RB   915.760877
5      6     3    Amon-Ra St. Brown   WR   901.225297
6      7     2        Ja'Marr Chase   WR   894.044258
7      8     1     Justin Jefferson   WR   886.920437
8      9     1       Saquon Barkley   RB   848.742022
9     10     2      Jonathan Taylor   RB   828.614707
10    11     3         Jahmyr Gibbs   RB   789.780674
11    12     4           A.J. Brown   WR   831.935804
12    13     4       Garrett Wilson   WR   796.124260
13    14     3       Kyren Williams   RB   740.818221
14    15     2   Travis Etienne Jr.   RB   720.363020
15    16     1           Puka Nacua   WR   767.973540
16    17     1           Josh Allen   QB   628.763554
17    18     2         Travi

# MIP: B. Optional: Value Over Replacement (VOR)

In [8]:
number_of_teams = 8
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

# Schritt 1: Replacement-Level pro Position bestimmen
replacement_rank = {pos: number_of_teams * req for pos, req in roster_requirements.items()}

replacement_value = {}
for pos in roster_requirements:
    pos_players = df[df['POS'] == pos].sort_values(by="Value", ascending=False).reset_index(drop=True)
    idx = replacement_rank[pos] - 1  # Nullbasiert
    if idx < len(pos_players):
        replacement_value[pos] = pos_players.loc[idx, "Value"]
    else:
        replacement_value[pos] = pos_players["Value"].min()

# Schritt 2: VOR berechnen
df["VOR"] = df.apply(lambda row: row["Value"] - replacement_value[row["POS"]], axis=1)

# Schritt 3: Snake-Draft nach VOR
teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["VOR"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren (jetzt als Dict, nicht nur Name)
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)
print("Draft Board:")
print(draft_df)

# Ausgabe: Spielerlisten pro Team
print("\nSpielerlisten pro Team:")
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Draft Board:
     Pick  Team               Player POS        Value         VOR
0       1     1  Christian McCaffrey  RB  1000.000000  537.912032
1       2     2       Bijan Robinson  RB   923.116346  461.028379
2       3     3          Breece Hall  RB   915.760877  453.672909
3       4     4          CeeDee Lamb  WR   968.506582  451.655248
4       5     5          Tyreek Hill  WR   956.953957  440.102623
..    ...   ...                  ...  ..          ...         ...
115   116     4         Joshua Karty   K     4.654131  -39.857711
116   117     5     Christopher Dunn   K     3.848776  -40.663066
117   118     6       Anders Carlson   K     3.247077  -41.264765
118   119     7            Joey Slye   K     3.214768  -41.297074
119   120     8           Josh Lambo   K     2.908843  -41.603000

[120 rows x 6 columns]

Spielerlisten pro Team:

Team 1:
                  Player  POS        Value         VOR
0    Christian McCaffrey   RB  1000.000000  537.912032
1            Jalen Hurts   

# MIP: Combination Value and VOR

In [9]:
number_of_teams = 12
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

# Neue Spalte für den kombinierten Score
Value_portion = 0.8
VOR_portion = 1 - Value_portion
df["CombinedScore"] = Value_portion * df["Value"] + VOR_portion * df["VOR"]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["CombinedScore"].idxmax()
    best_player = available.loc[best_idx]
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)

# Spielerlisten pro Team ausgeben
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Team 1:
                    Player  POS        Value         VOR  CombinedScore
0      Christian McCaffrey   RB  1000.000000  537.912032     907.582406
1             Drake London   WR   611.402366   94.551031     508.032099
2              Chris Olave   WR   604.109383   87.258048     500.739116
3            George Kittle   TE   328.900502   50.863201     273.293042
4   Anthony Richardson Sr.   QB   322.387836   62.628576     270.435984
5             Najee Harris   RB   247.585327 -214.502640     155.167734
6      San Francisco 49ers  DST   100.258844   58.673189      91.941713
7               Tyler Bass    K    26.043163  -18.468680      17.140795
8             Amari Cooper   WR   327.587528 -189.263807     224.217261
9           Tua Tagovailoa   QB   100.660682 -159.098578      48.708830
10              Jared Goff   QB   100.258844 -159.500416      48.306992
11             Gus Edwards   RB   105.188637 -356.899331      12.771043
12          Pat Freiermuth   TE    68.016837 -210.02046

In [10]:
draft_df

,Pick,Team,Player,POS,Value,VOR,CombinedScore
0,1,1,Christian McCaffrey,RB,1000.000000,537.912032,907.582406
1,2,2,CeeDee Lamb,WR,968.506582,451.655248,865.136315
2,3,3,Tyreek Hill,WR,956.953957,440.102623,853.583691
3,4,4,Bijan Robinson,RB,923.116346,461.028379,830.698753
4,5,5,Breece Hall,RB,915.760877,453.672909,823.343283
...,...,...,...,...,...,...,...
175,176,8,Cam Little,K,8.968830,-35.543012,0.066462
176,177,9,Mason Crosby,K,8.915179,-35.596664,0.012810
177,178,10,Chase McLaughlin,K,8.915179,-35.596664,0.012810
178,179,11,Tennessee Titans,DST,8.229747,-33.355908,-0.087384


# Local OPT

In [11]:
import pandas as pd
import pulp as pl

# Parameter
NUM_TEAMS = 8
MAX_TEAM_SIZE = 15
POS_REQ = {'QB':1, 'RB':2, 'WR':2, 'TE':1, 'K':1, 'DST':1}
TEAMS = list(range(1, NUM_TEAMS + 1))
PLAYERS = df_small.index.tolist()

# Snake-Draft-Reihenfolge generieren
draft_order = []
for rnd in range(MAX_TEAM_SIZE):
    draft_order += TEAMS if rnd % 2 == 0 else reversed(TEAMS)

# Modell erstellen
model = pl.LpProblem("SnakeDraft_MIP", pl.LpMaximize)

# Entscheidungsvariablen: x[p,i] = 1 wenn bei Pick p Spieler i gewählt wird
x = pl.LpVariable.dicts(
    "Pick",
    [(p, i) for p in range(len(draft_order)) for i in PLAYERS],
    cat=pl.LpBinary
)

# Zielfunktion: Gesamtwert maximieren
model += pl.lpSum(df_small.loc[i, "Value"] * x[p,i] for p,i in x)

# Constraints
# 1. Jeder Spieler maximal einmal gewählt
for i in PLAYERS:
    model += pl.lpSum(x[p,i] for p in range(len(draft_order))) <= 1

# 2. Jeder Pick wählt genau einen Spieler
for p in range(len(draft_order)):
    model += pl.lpSum(x[p,i] for i in PLAYERS) == 1

# 3. Positionsanforderungen pro Team
for t in TEAMS:
    team_picks = [p for p in range(len(draft_order)) if draft_order[p] == t]
    for pos, req in POS_REQ.items():
        pos_players = [i for i in PLAYERS if df_small.loc[i, "POS"] == pos]
        model += pl.lpSum(x[p,i] for p in team_picks for i in pos_players) >= req

# Modell lösen
solver = pl.PULP_CBC_CMD(msg=True)
model.solve(solver)

# Ergebnisse ausgeben
draft_results = []
for p in range(len(draft_order)):
    t = draft_order[p]
    for i in PLAYERS:
        if x[p,i].value() == 1:
            draft_results.append({
                'Pick': p+1,
                'Team': t,
                'Player': df_small.loc[i, 'Player'],
                'POS': df_small.loc[i, 'POS'],
                'Value': df_small.loc[i, 'Value']
            })

draft_df = pd.DataFrame(draft_results).sort_values('Pick')
print("\nDraft Verlauf:")
print(draft_df)

# Team-Roster prüfen
for t in TEAMS:
    team_df = draft_df[draft_df['Team'] == t]
    print(f"\nTeam {t} Roster:")
    print(team_df[['Player', 'POS', 'Value']].reset_index(drop=True))



Draft Verlauf:
     Pick  Team               Player  POS       Value
0       1     1          Jayden Reed   WR  178.351314
1       2     2          Zamir White   RB  269.280956
2       3     3         Dak Prescott   QB  259.759260
3       4     4  Pittsburgh Steelers  DST   42.766508
4       5     5         Tyjae Spears   RB  137.518063
..    ...   ...                  ...  ...         ...
115   116     4           DK Metcalf   WR  447.535238
116   117     5   Kenneth Walker III   RB  445.748673
117   118     6        D'Andre Swift   RB  303.613430
118   119     7       Christian Kirk   WR  229.466094
119   120     8       Brandon Aubrey    K  105.610234

[120 rows x 5 columns]

Team 1 Roster:
                Player  POS       Value
0          Jayden Reed   WR  178.351314
1       Saquon Barkley   RB  848.742022
2             DJ Moore   WR  426.560956
3            Tank Dell   WR  278.037300
4          David Njoku   TE  189.379943
5     Devin Singletary   RB  154.432218
6        DeVonta

#

In [35]:
import pandas as pd
import numpy as np

# Konfiguration
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15
number_of_teams = 12
position_priority = {'RB': 1, 'WR': 2, 'QB': 3, 'TE': 4, 'K': 5, 'DST': 6}

# ADP-Spalte hinzufügen (falls nicht vorhanden)
if 'ADP' not in df.columns:
    np.random.seed(42)
    df['ADP'] = df['AVG'] + np.random.normal(0, 2, size=len(df))

# Snake Draft Reihenfolge generieren
teams = list(range(1, number_of_teams + 1))
draft_order = []
for r in range(max_team_size):
    draft_order += teams if r % 2 == 0 else teams[::-1]

# Initialisierung der Roster
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}
available = df.copy()
draft_results = []

def get_needs(team):
    needs = []
    for pos, req in roster_requirements.items():
        if roster_counts[team][pos] < req:
            needs.append(pos)
    return needs if needs else list(roster_requirements.keys())

# Draft Simulation
for pick_num, team in enumerate(draft_order):
    if available.empty:
        break

    # Aktuelle Bedarfe ermitteln
    needs = get_needs(team)

    # Kandidaten filtern und sortieren
    candidates = available[available['POS'].isin(needs)]
    if candidates.empty:
        candidates = available

    candidates = candidates.copy()
    candidates['pos_priority'] = candidates['POS'].map(position_priority)
    candidates = candidates.sort_values(by=['ADP', 'pos_priority'])

    # Spieler auswählen
    selected = candidates.iloc[0]

    # Roster aktualisieren
    pos = selected['POS']
    rosters[team].append(selected)
    roster_counts[team][pos] += 1
    draft_results.append({
        'Pick': pick_num + 1,
        'Team': team,
        'Player': selected['Player'],
        'POS': pos,
        'ADP': selected['ADP'],
        'Value': selected['Value']
    })

    # Spieler aus Available entfernen
    available = available.drop(selected.name)

# Ergebnisse analysieren
draft_df = pd.DataFrame(draft_results)

# Team-Roster anzeigen
for team in teams:  # Zeige nur die ersten 3 Teams zur Demonstration
    print(f"\nTeam {team} Roster:")
    team_players = pd.DataFrame(rosters[team])
    print(team_players[['Player', 'POS', 'ADP', 'Value']].reset_index(drop=True))

print(f"\nVerbleibende verfügbare Spieler: {len(available)}")



Team 1 Roster:
                 Player  POS         ADP        Value
0   Christian McCaffrey   RB    1.993428  1000.000000
1          Nico Collins   WR   25.898013   580.421915
2         De'Von Achane   RB   26.311235   589.783358
3            DK Metcalf   WR   42.676933   447.535238
4           C.J. Stroud   QB   45.073920   385.968314
5         Justin Tucker    K  107.714571   116.717359
6        Dallas Goedert   TE  109.277429   113.949490
7        Houston Texans  DST  185.654366    25.732513
8        George Pickens   WR   57.921565   309.746828
9    Jaxon Smith-Njigba   WR   98.395894   138.622620
10         Tyjae Spears   RB   99.544676   137.518063
11          Blake Corum   RB  130.805589    78.866400
12        Khalil Shakir   WR  130.867405    72.802863
13    Jaleel McLaughlin   RB  165.731549    39.955058
14       Baker Mayfield   QB  167.913107    36.443214

Team 2 Roster:
                 Player  POS         ADP       Value
0           CeeDee Lamb   WR    2.323471  968.50658